# 🌾 GrassClover-Style Generation with Stable Diffusion

Este notebook implementa geração de imagens sintéticas de pastagens brasileiras usando **Stable Diffusion**, seguindo o estilo visual do **GrassClover Dataset** (Skovsen et al., CVPR 2019).

## 🎯 Objetivos:
- Gerar imagens **top-down** de pastagens com Stable Diffusion
- Adaptar para **gramíneas brasileiras** (Brachiaria, Panicum, Cynodon)
- Seguir **metodologia GrassClover** (densidade, perspectiva, resolução)
- **Ultra-compatível** com Google Colab (debugging extensivo)

## 📚 Referência:
- **Paper**: Skovsen et al. "The GrassClover Image Dataset for Semantic and Hierarchical Species Understanding in Agriculture" (CVPR Workshops, 2019)
- **Adaptação**: Espécies temperadas → Tropicais brasileiras

---

## 🚀 Setup Ultra-Compatível para Colab

**IMPORTANTE**: Este notebook foi desenvolvido para máxima compatibilidade com Google Colab Free/Pro.

In [ ]:
# 🔍 VERIFICAÇÃO INICIAL DO AMBIENTE
print("=" * 60)
print("🌾 GRASSCLOVER STABLE DIFFUSION GENERATOR")
print("=" * 60)

import sys
import platform
from datetime import datetime

print(f"📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python: {sys.version.split()[0]}")
print(f"💻 Platform: {platform.system()} {platform.release()}")
print(f"📍 Working Directory: {sys.path[0] if sys.path else 'Unknown'}")

# Detectar se estamos no Colab
try:
    import google.colab
    IN_COLAB = True
    print("🔥 Ambiente: Google Colab (DETECTADO)")
except ImportError:
    IN_COLAB = False
    print("💻 Ambiente: Local/Jupyter (DETECTADO)")

print("\n" + "=" * 60)

In [ ]:
# 🧠 VERIFICAÇÃO AVANÇADA DE HARDWARE (GPU/TPU/CPU)
print("🔍 Verificando hardware disponível...\n")

# Variáveis de estado
device = None
device_type = "unknown"
hardware_info = {}

# Tentar importar torch primeiro
try:
    import torch
    TORCH_AVAILABLE = True
    print(f"✅ PyTorch: {torch.__version__}")
    hardware_info['pytorch_version'] = torch.__version__
except ImportError:
    TORCH_AVAILABLE = False
    print("❌ PyTorch não encontrado - será instalado")

if TORCH_AVAILABLE:
    # 1. VERIFICAR TPU (prioridade alta no Colab)
    try:
        import torch_xla
        import torch_xla.core.xla_model as xm
        
        # Detectar TPU
        if xm.xrt_world_size() > 1:
            device = xm.xla_device()
            device_type = "tpu"
            print(f"🔥 TPU DETECTADO: {device}")
            print(f"🚀 TPU cores: {xm.xrt_world_size()}")
            hardware_info.update({
                'device_type': 'tpu',
                'tpu_cores': xm.xrt_world_size(),
                'device': str(device)
            })
        else:
            print("⚠️  TPU configurado mas não disponível")
            
    except ImportError:
        print("📝 torch_xla não encontrado (normal se não usar TPU)")
    except Exception as e:
        print(f"⚠️  Erro ao verificar TPU: {e}")
    
    # 2. VERIFICAR CUDA/GPU (se TPU não disponível)
    if device is None:
        cuda_available = torch.cuda.is_available()
        print(f"🔥 CUDA disponível: {cuda_available}")
        
        if cuda_available:
            gpu_count = torch.cuda.device_count()
            print(f"🚀 Número de GPUs: {gpu_count}")
            
            for i in range(gpu_count):
                gpu_name = torch.cuda.get_device_name(i)
                gpu_memory = torch.cuda.get_device_properties(i).total_memory
                gpu_memory_gb = gpu_memory / (1024**3)
                print(f"  GPU {i}: {gpu_name}")
                print(f"  Memória: {gpu_memory_gb:.1f} GB")
            
            device = torch.device("cuda")
            device_type = "gpu"
            hardware_info.update({
                'device_type': 'gpu',
                'gpu_count': gpu_count,
                'gpu_name': torch.cuda.get_device_name(0),
                'gpu_memory_gb': torch.cuda.get_device_properties(0).total_memory / (1024**3),
                'device': str(device)
            })
            
            # Limpeza inicial de memória GPU
            torch.cuda.empty_cache()
            print("🧹 Cache GPU limpo")
            
        else:
            print("⚠️  CUDA não disponível")
    
    # 3. FALLBACK PARA CPU
    if device is None:
        device = torch.device("cpu")
        device_type = "cpu"
        hardware_info.update({
            'device_type': 'cpu',
            'device': str(device)
        })
        print("💻 Usando CPU (fallback)")

    # Status final do hardware
    print(f"\n🎯 DEVICE CONFIGURADO: {device} ({device_type.upper()})")
    
    # Instruções específicas para problemas
    if device_type == "cpu" and IN_COLAB:
        print(f"\n🚨 IMPORTANTE: Você está usando CPU no Colab!")
        print(f"Para ativar aceleração de hardware:")
        print(f"1. Runtime → Change runtime type")
        print(f"2. Hardware accelerator: GPU ou TPU") 
        print(f"3. Save → Connect (reconectar)")
        print(f"4. Re-executar este notebook")
        
    elif device_type == "tpu":
        print(f"✅ TPU configurado! Otimizado para treinamento paralelo.")
        
    elif device_type == "gpu":
        print(f"✅ GPU configurado! Otimizado para inferência rápida.")

else:
    device = None
    device_type = "none"
    print("\n⏳ PyTorch será instalado na próxima célula")

print(f"\n📊 Hardware Info: {hardware_info}")
print("\n" + "=" * 60)

In [ ]:
# 🔐 AUTENTICAÇÃO HUGGINGFACE 
print("🔐 Configurando autenticação HuggingFace...\n")

try:
    from huggingface_hub import notebook_login
    
    print("📋 Para usar Stable Diffusion 3.5 Large, você precisa fazer login no HuggingFace")
    print("💡 Aceite os termos em: https://huggingface.co/stabilityai/stable-diffusion-3.5-large")
    print("⏳ Executando login...")
    
    # Login interativo no notebook
    notebook_login()
    
    print("✅ Login realizado com sucesso!")
    HUGGINGFACE_LOGGED_IN = True
    
except Exception as e:
    print(f"⚠️  Erro no login: {e}")
    print("🔄 Continuando sem autenticação (fallback para SD1.5)")
    HUGGINGFACE_LOGGED_IN = False

print("=" * 60)

In [ ]:
# 🚀 STABLE DIFFUSION 3.5 LARGE - CONFIGURAÇÃO CORRIGIDA
print("🚀 Configurando Stable Diffusion 3.5 Large...\n")

# 🎯 VERSÃO CORRIGIDA: SD3.5 com fallback robusto
def load_stable_diffusion_pipeline_advanced():
    """
    Carrega SD3.5 Large com fallback automático e tratamento de erros robusto
    """
    print("🔄 Tentando carregar Stable Diffusion 3.5 Large...")
    
    # Verificar se temos login do HuggingFace
    if not HUGGINGFACE_LOGGED_IN:
        print("⚠️  HuggingFace login não realizado, tentando mesmo assim...")
    
    try:
        # Imports específicos para SD3.5
        from diffusers import StableDiffusion3Pipeline
        import torch
        
        print("📦 Imports SD3.5 bem-sucedidos")
        
        # Configurações mais conservadoras para SD3.5
        load_kwargs = {
            "torch_dtype": torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            "device_map": "auto" if torch.cuda.is_available() else None,
            "low_cpu_mem_usage": True,
        }
        
        print(f"🔧 Tentando carregar com parâmetros: {load_kwargs}")
        
        # Tentar carregar SD3.5 com tratamento mais robusto
        pipe = StableDiffusion3Pipeline.from_pretrained(
            "stabilityai/stable-diffusion-3.5-large", 
            **load_kwargs
        )
        
        print("✅ Stable Diffusion 3.5 Large carregado com sucesso!")
        
        # Mover para device se necessário (já pode estar no device certo com device_map)
        if not torch.cuda.is_available() or load_kwargs.get("device_map") is None:
            pipe = pipe.to(device)
        
        return pipe, "3.5"
        
    except ImportError as e:
        print(f"❌ Erro de import SD3.5: {e}")
        print("💡 Pode ser necessário atualizar diffusers: pip install --upgrade diffusers")
        
    except Exception as e:
        print(f"⚠️  SD3.5 falhou: {str(e)[:200]}...")
        print(f"🔍 Tipo do erro: {type(e).__name__}")
        
        # Erros comuns e soluções
        if "get() takes 1 positional argument" in str(e):
            print("💡 Erro de compatibilidade - tentando parâmetros alternativos...")
            
            try:
                # Tentativa com parâmetros mais simples
                simple_kwargs = {
                    "torch_dtype": torch.bfloat16 if torch.cuda.is_available() else torch.float32,
                }
                
                pipe = StableDiffusion3Pipeline.from_pretrained(
                    "stabilityai/stable-diffusion-3.5-large", 
                    **simple_kwargs
                )
                pipe = pipe.to(device)
                print("✅ SD3.5 carregado com parâmetros simplificados!")
                return pipe, "3.5"
                
            except Exception as e2:
                print(f"❌ Tentativa simplificada também falhou: {e2}")
                
        elif "requires you to be logged in" in str(e) or "authentication" in str(e):
            print("🔐 Erro de autenticação - verifique seu login no HuggingFace")
            
        elif "out of memory" in str(e).lower() or "oom" in str(e).lower():
            print("💾 Erro de memória - SD3.5 requer muita VRAM")
    
    # Fallback para SD1.5
    print("🔄 Iniciando fallback para Stable Diffusion 1.5...")
    
    try:
        from diffusers import StableDiffusionPipeline
        
        # Parâmetros para SD1.5
        fallback_kwargs = {
            "torch_dtype": TORCH_DTYPE,
            "low_cpu_mem_usage": True,
            "use_safetensors": True,
        }
        
        pipe = StableDiffusionPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5", 
            **fallback_kwargs
        )
        
        pipe = pipe.to(device)
        print("✅ Fallback SD1.5 carregado com sucesso!")
        return pipe, "1.5"
        
    except Exception as e2:
        print(f"❌ Fallback SD1.5 também falhou: {e2}")
        return None, None

# Carregar pipeline com função corrigida
pipe, sd_version = load_stable_diffusion_pipeline_advanced()

if pipe is not None:
    print(f"\n🎯 Pipeline Stable Diffusion {sd_version} carregado!")
    
    # Configurar scheduler se necessário
    try:
        if sd_version == "1.5":
            from diffusers import DPMSolverMultistepScheduler
            pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
            print("🔧 Scheduler DPMSolver aplicado (SD1.5)")
        else:
            print("🔧 Usando scheduler nativo do SD3.5")
    except Exception as e:
        print(f"⚠️  Mantendo scheduler padrão: {e}")
    
    # Aplicar otimizações baseadas na versão
    if sd_version == "3.5":
        print(f"🎯 Configuração SD3.5:")
        print(f"  • Dtype: torch.bfloat16")
        print(f"  • Steps recomendados: 28")
        print(f"  • Guidance scale: 4.0")
        
        GENERATION_PARAMS_CURRENT = {
            'width': 512,
            'height': 512,
            'num_inference_steps': 28,
            'guidance_scale': 4.0,
            'num_images_per_prompt': 1,
            'eta': 0.0,
            'generator_seed': 42
        }
        
    else:  # SD1.5
        print(f"🎯 Configuração SD1.5:")
        print(f"  • Dtype: {TORCH_DTYPE}")
        print(f"  • Steps: 25")
        print(f"  • Guidance scale: 7.5")
        
        GENERATION_PARAMS_CURRENT = {
            'width': 512,
            'height': 512,
            'num_inference_steps': 25,
            'guidance_scale': 7.5,
            'num_images_per_prompt': 1,
            'eta': 0.0,
            'generator_seed': 42
        }
    
    # Aplicar otimizações de memória
    try:
        if hasattr(pipe, 'enable_attention_slicing'):
            pipe.enable_attention_slicing()
            print("✅ Attention slicing ativado")
            
        if hasattr(pipe, 'enable_memory_efficient_attention') and device.type == "cuda":
            pipe.enable_memory_efficient_attention()  
            print("✅ Memory efficient attention ativado")
    except Exception as e:
        print(f"⚠️  Algumas otimizações falharam: {e}")
    
    PIPELINE_READY = True
    print(f"\n✅ Pipeline pronto para uso!")
    print(f"🎯 Device: {device}")
    
else:
    PIPELINE_READY = False
    print("❌ Falha total - nenhuma versão do Stable Diffusion foi carregada!")
    print("💡 Possíveis soluções:")
    print("  1. Verificar login no HuggingFace")
    print("  2. Atualizar diffusers: pip install --upgrade diffusers")
    print("  3. Reiniciar runtime se necessário")

print("=" * 60)


In [ ]:
# 📦 INSTALAÇÃO DE DEPENDÊNCIAS ESSENCIAIS
print("📦 Instalando dependências essenciais...\n")

# Lista de pacotes essenciais
essential_packages = [
    "torch",
    "torchvision", 
    "diffusers",
    "transformers",
    "accelerate",
    "pillow",
    "matplotlib",
    "numpy"
]

# Função para instalar com debug
def install_package(package_name, quiet=True):
    """Instala pacote com debugging"""
    import subprocess
    
    print(f"⏳ Instalando {package_name}...")
    
    try:
        cmd = ["pip", "install", package_name]
        if quiet:
            cmd.append("--quiet")
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            print(f"✅ {package_name} instalado com sucesso")
            return True
        else:
            print(f"❌ Erro ao instalar {package_name}:")
            print(f"STDOUT: {result.stdout}")
            print(f"STDERR: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"⏰ Timeout ao instalar {package_name}")
        return False
    except Exception as e:
        print(f"💥 Exceção ao instalar {package_name}: {e}")
        return False

# Instalar apenas se necessário
for package in essential_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✅ {package} já disponível")
    except ImportError:
        success = install_package(package)
        if not success:
            print(f"🚨 FALHA CRÍTICA: Não foi possível instalar {package}")
            print(f"📋 DEBUG INFO para copy/paste:")
            print(f"Package: {package}")
            print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")
            print(f"Python: {sys.version}")
            break

print(f"\n🎉 Instalação concluída!")
print("=" * 60)

In [ ]:
# 🔄 REIMPORTAÇÃO E VERIFICAÇÃO FINAL COM CONFIGURAÇÕES OTIMIZADAS
print("🔄 Verificando importações finais e configurando hardware...\n")

# Imports essenciais com debugging
try:
    import torch
    import torchvision.transforms as transforms
    print(f"✅ PyTorch: {torch.__version__}")
    
    # Reconfigurar device após instalação se necessário
    if not 'device' in locals() or device is None:
        if torch.cuda.is_available():
            device = torch.device("cuda")
            device_type = "gpu"
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            print(f"🚀 GPU: {gpu_name} ({gpu_memory:.1f} GB)")
            
            # Configuração otimizada para GPU
            torch.backends.cudnn.benchmark = True
            torch.cuda.empty_cache()
            
        else:
            device = torch.device("cpu")
            device_type = "cpu"
            print("💻 Device: CPU")
    
    # Configurar dtype baseado no hardware
    if device_type == "gpu":
        TORCH_DTYPE = torch.bfloat16  # GPU: usar half precision
        print(f"🔢 Dtype: {TORCH_DTYPE} (GPU otimizado)")
    elif device_type == "tpu":
        TORCH_DTYPE = torch.float32  # TPU: full precision recomendado
        print(f"🔢 Dtype: {TORCH_DTYPE} (TPU otimizado)")
    else:
        TORCH_DTYPE = torch.float32  # CPU: full precision
        print(f"🔢 Dtype: {TORCH_DTYPE} (CPU padrão)")
        
except ImportError as e:
    print(f"❌ Erro PyTorch: {e}")
    device = None
    device_type = "none"
    TORCH_DTYPE = None

try:
    from diffusers import StableDiffusion3Pipeline, DPMSolverMultistepScheduler
    print(f"✅ Diffusers importado")
    DIFFUSERS_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erro Diffusers: {e}")
    DIFFUSERS_AVAILABLE = False

try:
    from PIL import Image, ImageEnhance, ImageFilter
    import matplotlib.pyplot as plt
    import numpy as np
    print(f"✅ PIL, Matplotlib, NumPy importados")
    BASIC_LIBS_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erro bibliotecas básicas: {e}")
    BASIC_LIBS_AVAILABLE = False

# Configuração de ambiente Hugging Face
print(f"\n🤗 Configurando Hugging Face...")
try:
    import os
    # Desabilitar telemetria para evitar warnings
    os.environ["DISABLE_TELEMETRY"] = "1"
    os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"
    
    # Configurar cache offline se necessário
    if IN_COLAB:
        os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Permitir downloads no Colab
        os.environ["HF_HUB_OFFLINE"] = "0"
    
    print(f"✅ Variáveis HF configuradas (telemetria desabilitada)")
except Exception as e:
    print(f"⚠️  Aviso na configuração HF: {e}")

# Status final
ALL_READY = device is not None and DIFFUSERS_AVAILABLE and BASIC_LIBS_AVAILABLE

print(f"\n{'🎯 SISTEMA PRONTO!' if ALL_READY else '🚨 PROBLEMAS DETECTADOS!'}")
if ALL_READY:
    print(f"✅ Device: {device} ({device_type})")
    print(f"✅ Dtype: {TORCH_DTYPE}")
    print(f"✅ Todas as bibliotecas carregadas")
else:
    print("\n📋 DEBUG COPY/PASTE INFO:")
    print(f"Device: {device}")
    print(f"Device type: {device_type if 'device_type' in locals() else 'unknown'}")
    print(f"Diffusers: {DIFFUSERS_AVAILABLE}")
    print(f"Basic libs: {BASIC_LIBS_AVAILABLE}")
    print(f"In Colab: {IN_COLAB}")
    
    if device_type == "cpu" and IN_COLAB:
        print(f"\n🔧 AÇÃO NECESSÁRIA:")
        print(f"1. Runtime → Change runtime type")
        print(f"2. Hardware accelerator: GPU")
        print(f"3. Save e reconectar")

print("=" * 60)

## 🎨 Pipeline Stable Diffusion para GrassClover

Configuração do pipeline otimizado para geração de pastagens no estilo GrassClover.

# 🎨 CONFIGURAÇÃO AVANÇADA DO PIPELINE STABLE DIFFUSION
print("🎨 Configurando Stable Diffusion Pipeline...\n")

# Parâmetros do pipeline baseados no hardware detectado
MODEL_ID = "runwayml/stable-diffusion-v1-5"  # Modelo base confiável
LOW_CPU_MEM_USAGE = True
ENABLE_ATTENTION_SLICING = True

# Configurações específicas por hardware
if device_type == "gpu":
    ENABLE_MODEL_CPU_OFFLOAD = False
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = False
    USE_TORCH_COMPILE = True
elif device_type == "tpu":
    ENABLE_MODEL_CPU_OFFLOAD = True
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = False
    USE_TORCH_COMPILE = False  # TPU pode ter problemas com compile
else:  # CPU
    ENABLE_MODEL_CPU_OFFLOAD = True
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = True
    USE_TORCH_COMPILE = False

print(f"📦 Modelo: {MODEL_ID}")
print(f"🔢 Dtype: {TORCH_DTYPE}")
print(f"💾 Low CPU mem: {LOW_CPU_MEM_USAGE}")
print(f"⚡ Attention slicing: {ENABLE_ATTENTION_SLICING}")
print(f"🏃 Model CPU offload: {ENABLE_MODEL_CPU_OFFLOAD}")
print(f"🔥 Hardware: {device_type.upper()}")

# Função para carregar pipeline com debugging e bypass de autenticação
def load_stable_diffusion_pipeline():
    """Carrega pipeline com máximo debugging e configurações otimizadas"""
    try:
        print("⏳ Carregando modelo...")
        
        # Configurações para bypass de problemas de autenticação
        load_kwargs = {
            "torch_dtype": TORCH_DTYPE,
            "low_cpu_mem_usage": LOW_CPU_MEM_USAGE,
            "use_safetensors": True,
        }
        
        # Configurações específicas para resolver problemas HF
        try:
            # Tentar com autenticação padrão primeiro
            pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, **load_kwargs)
            print("✅ Modelo carregado com autenticação padrão")
        except Exception as auth_error:
            print(f"⚠️  Problema de autenticação: {str(auth_error)[:100]}...")
            print("🔄 Tentando download forçado...")
            
            # Bypass de problemas de autenticação
            load_kwargs["use_auth_token"] = False
            load_kwargs["force_download"] = False
            load_kwargs["resume_download"] = True
            
            pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, **load_kwargs)
            print("✅ Modelo carregado com bypass de autenticação")
        
        # Mover para device
        pipe = pipe.to(device)
        print(f"🚀 Pipeline movido para {device}")
        
        # Otimizações baseadas no hardware
        if ENABLE_ATTENTION_SLICING:
            pipe.enable_attention_slicing()
            print("⚡ Attention slicing habilitado")
        
        if ENABLE_MODEL_CPU_OFFLOAD and device_type != "cpu":
            try:
                pipe.enable_model_cpu_offload()
                print("💾 Model CPU offload habilitado")
            except Exception as e:
                print(f"⚠️  CPU offload falhou: {e}")
        
        if ENABLE_SEQUENTIAL_CPU_OFFLOAD and device_type == "cpu":
            try:
                pipe.enable_sequential_cpu_offload()
                print("🔄 Sequential CPU offload habilitado")
            except Exception as e:
                print(f"⚠️  Sequential offload falhou: {e}")
        
        # Scheduler otimizado
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
        print("🔧 Scheduler otimizado (DPMSolver)")
        
        # Configurações de memória específicas
        if device_type == "gpu":
            torch.cuda.empty_cache()
            allocated = torch.cuda.memory_allocated() / (1024**3)
            cached = torch.cuda.memory_reserved() / (1024**3)
            print(f"💾 GPU Memory - Allocated: {allocated:.2f}GB, Cached: {cached:.2f}GB")
            
            # Verificar se há memória suficiente
            if allocated > 10.0:  # >10GB pode causar problemas
                print(f"⚠️  Alta utilização de memória GPU!")
                
        elif device_type == "tpu":
            print("🔥 TPU configurado - memória gerenciada automaticamente")
            
        else:  # CPU
            print("💻 CPU mode - sem monitoramento de GPU memory")
        
        print("\n✅ Pipeline configurado com sucesso!")
        return pipe
        
    except Exception as e:
        print(f"💥 ERRO ao carregar pipeline: {e}")
        print(f"\n📋 DEBUG COPY/PASTE:")
        print(f"Model: {MODEL_ID}")
        print(f"Device: {device} ({device_type})")
        print(f"Dtype: {TORCH_DTYPE}")
        print(f"Hardware info: {hardware_info if 'hardware_info' in locals() else 'N/A'}")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {e}")
        
        # Sugestões baseadas no tipo de erro
        error_str = str(e).lower()
        if "authentication" in error_str or "token" in error_str:
            print(f"\n💡 SOLUÇÃO PARA ERRO DE TOKEN:")
            print(f"1. Ignore o warning - o modelo deve funcionar mesmo assim")
            print(f"2. Ou configure HF_TOKEN manualmente se necessário")
            
        elif "memory" in error_str or "cuda" in error_str:
            print(f"\n💡 SOLUÇÃO PARA ERRO DE MEMÓRIA:")
            print(f"1. Reduzir batch_size")
            print(f"2. Usar torch.float16 se estiver usando float32")
            print(f"3. Fechar outros notebooks no Colab")
            
        elif "module" in error_str or "import" in error_str:
            print(f"\n💡 SOLUÇÃO PARA ERRO DE MÓDULO:")
            print(f"1. Re-executar células de instalação")
            print(f"2. Restart runtime se necessário")
            
        return None

# Carregar pipeline
if ALL_READY:
    pipe = load_stable_diffusion_pipeline()
    PIPELINE_READY = pipe is not None
else:
    pipe = None
    PIPELINE_READY = False
    print("❌ Sistema não está pronto para carregar pipeline")

print("=" * 60)

In [ ]:
# 🎨 CONFIGURAÇÃO AVANÇADA DO PIPELINE STABLE DIFFUSION
print("🎨 Configurando Stable Diffusion Pipeline...\n")

# Parâmetros do pipeline baseados no hardware detectado
MODEL_ID = "stabilityai/stable-diffusion-3.5-large"  # Modelo SD 3.5 Large - Estado da Arte
LOW_CPU_MEM_USAGE = True
ENABLE_ATTENTION_SLICING = True

# Configurações específicas por hardware
if device_type == "gpu":
    ENABLE_MODEL_CPU_OFFLOAD = False
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = False
    USE_TORCH_COMPILE = True
elif device_type == "tpu":
    ENABLE_MODEL_CPU_OFFLOAD = True
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = False
    USE_TORCH_COMPILE = False  # TPU pode ter problemas com compile
else:  # CPU
    ENABLE_MODEL_CPU_OFFLOAD = True
    ENABLE_SEQUENTIAL_CPU_OFFLOAD = True
    USE_TORCH_COMPILE = False

print(f"📦 Modelo: {MODEL_ID}")
print(f"🔢 Dtype: {TORCH_DTYPE}")
print(f"💾 Low CPU mem: {LOW_CPU_MEM_USAGE}")
print(f"⚡ Attention slicing: {ENABLE_ATTENTION_SLICING}")
print(f"🏃 Model CPU offload: {ENABLE_MODEL_CPU_OFFLOAD}")
print(f"🔥 Hardware: {device_type.upper()}")

# Função para carregar pipeline com debugging e bypass de autenticação
def load_stable_diffusion_pipeline():
    # """Carrega pipeline com máximo debugging e configurações otimizadas"""
    try:
        print("⏳ Carregando modelo...")
        
        # Configurações para bypass de problemas de autenticação
        load_kwargs = {
            "torch_dtype": TORCH_DTYPE,
            "low_cpu_mem_usage": LOW_CPU_MEM_USAGE,
            "use_safetensors": True,
        }
        
        # Configurações específicas para resolver problemas HF
        try:
            # Tentar com autenticação padrão primeiro
            pipe = StableDiffusion3Pipeline.from_pretrained(MODEL_ID, **load_kwargs)
            print("✅ Modelo carregado com autenticação padrão")
        except Exception as auth_error:
            print(f"⚠️  Problema de autenticação: {str(auth_error)[:100]}...")
            print("🔄 Tentando download forçado...")
            
            # Bypass de problemas de autenticação
            load_kwargs["use_auth_token"] = False
            load_kwargs["force_download"] = False
            load_kwargs["resume_download"] = True
            
            pipe = StableDiffusion3Pipeline.from_pretrained(MODEL_ID, **load_kwargs)
            print("✅ Modelo carregado com bypass de autenticação")
        
        # Mover para device
        pipe = pipe.to(device)
        print(f"🚀 Pipeline movido para {device}")
        
        # Otimizações baseadas no hardware
        if ENABLE_ATTENTION_SLICING:
            pipe.enable_attention_slicing()
            print("⚡ Attention slicing habilitado")
        
        if ENABLE_MODEL_CPU_OFFLOAD and device_type != "cpu":
            try:
                pipe.enable_model_cpu_offload()
                print("💾 Model CPU offload habilitado")
            except Exception as e:
                print(f"⚠️  CPU offload falhou: {e}")
        
        if ENABLE_SEQUENTIAL_CPU_OFFLOAD and device_type == "cpu":
            try:
                pipe.enable_sequential_cpu_offload()
                print("🔄 Sequential CPU offload habilitado")
            except Exception as e:
                print(f"⚠️  Sequential offload falhou: {e}")
        
        # Scheduler otimizado
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
        print("🔧 Scheduler otimizado (DPMSolver)")
        
        # Configurações de memória específicas
        if device_type == "gpu":
            torch.cuda.empty_cache()
            allocated = torch.cuda.memory_allocated() / (1024**3)
            cached = torch.cuda.memory_reserved() / (1024**3)
            print(f"💾 GPU Memory - Allocated: {allocated:.2f}GB, Cached: {cached:.2f}GB")
            
            # Verificar se há memória suficiente
            if allocated > 10.0:  # >10GB pode causar problemas
                print(f"⚠️  Alta utilização de memória GPU!")
                
        elif device_type == "tpu":
            print("🔥 TPU configurado - memória gerenciada automaticamente")
            
        else:  # CPU
            print("💻 CPU mode - sem monitoramento de GPU memory")
        
        print("\\n✅ Pipeline configurado com sucesso!")
        return pipe
        
    except Exception as e:
        print(f"💥 ERRO ao carregar pipeline: {e}")
        print(f"\\n📋 DEBUG COPY/PASTE:")
        print(f"Model: {MODEL_ID}")
        print(f"Device: {device} ({device_type})")
        print(f"Dtype: {TORCH_DTYPE}")
        print(f"Hardware info: {hardware_info if 'hardware_info' in locals() else 'N/A'}")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {e}")
        
        # Sugestões baseadas no tipo de erro
        error_str = str(e).lower()
        if "authentication" in error_str or "token" in error_str:
            print(f"\\n💡 SOLUÇÃO PARA ERRO DE TOKEN:")
            print(f"1. Ignore o warning - o modelo deve funcionar mesmo assim")
            print(f"2. Ou configure HF_TOKEN manualmente se necessário")
            
        elif "memory" in error_str or "cuda" in error_str:
            print(f"\\n💡 SOLUÇÃO PARA ERRO DE MEMÓRIA:")
            print(f"1. Reduzir batch_size")
            print(f"2. Usar torch.float16 se estiver usando float32")
            print(f"3. Fechar outros notebooks no Colab")
            
        elif "module" in error_str or "import" in error_str:
            print(f"\\n💡 SOLUÇÃO PARA ERRO DE MÓDULO:")
            print(f"1. Re-executar células de instalação")
            print(f"2. Restart runtime se necessário")
            
        return None

# Carregar pipeline
if ALL_READY:
    pipe = load_stable_diffusion_pipeline()
    PIPELINE_READY = pipe is not None
else:
    pipe = None
    PIPELINE_READY = False
    print("❌ Sistema não está pronto para carregar pipeline")

print("=" * 60)

In [ ]:
# 📚 DOWNLOAD DO DATASET GRASSCLOVER ORIGINAL
print("📚 Baixando dataset GrassClover original do Kaggle...\n")

# Inicializar variável globalmente
GRASSCLOVER_DATASET_PATH = None

try:
    # Instalar kagglehub se necessário
    try:
        import kagglehub
        print("✅ kagglehub já disponível")
    except ImportError:
        print("📦 Instalando kagglehub...")
        import subprocess
        result = subprocess.run(["pip", "install", "kagglehub", "--quiet"], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            import kagglehub
            print("✅ kagglehub instalado com sucesso")
        else:
            print(f"❌ Erro ao instalar kagglehub: {result.stderr}")
            raise ImportError("kagglehub installation failed")
    
    # Download do dataset
    print("⏳ Fazendo download do GrassClover dataset...")
    print("💡 Isso pode demorar alguns minutos na primeira vez...")
    
    dataset_path = kagglehub.dataset_download("usharengaraju/grassclover-dataset")
    
    if dataset_path and os.path.exists(dataset_path):
        GRASSCLOVER_DATASET_PATH = dataset_path
        print(f"✅ Dataset baixado com sucesso!")
        print(f"📁 Localização: {dataset_path}")
        
        # Explorar estrutura do dataset
        print(f"\n📋 Estrutura do dataset:")
        
        total_files = 0
        for root, dirs, files in os.walk(dataset_path):
            level = root.replace(dataset_path, '').count(os.sep)
            indent = '  ' * level
            folder_name = os.path.basename(root) if root != dataset_path else 'grassclover-dataset'
            
            # Contar apenas arquivos de imagem
            image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            if image_files:
                print(f"{indent}{folder_name}/ ({len(image_files)} imagens)")
                total_files += len(image_files)
                
                # Mostrar alguns exemplos de nomes
                for file in image_files[:3]:
                    print(f"{indent}  • {file}")
                if len(image_files) > 3:
                    print(f"{indent}  • ... e mais {len(image_files)-3} imagens")
            elif dirs:
                print(f"{indent}{folder_name}/")
        
        print(f"\n📊 Total: {total_files} imagens encontradas")
        
        if total_files == 0:
            print("⚠️  Nenhuma imagem encontrada no dataset baixado")
            GRASSCLOVER_DATASET_PATH = None
    else:
        print("❌ Download retornou path inválido")
        GRASSCLOVER_DATASET_PATH = None
    
except Exception as e:
    print(f"❌ Erro ao baixar dataset: {e}")
    print(f"\n📋 DEBUG INFO:")
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)[:200]}...")
    
    # Manter variável definida como None
    GRASSCLOVER_DATASET_PATH = None
    
    print(f"\n💡 SOLUÇÕES POSSÍVEIS:")
    print(f"1. Verificar conectividade com internet")
    print(f"2. Tentar novamente em alguns minutos")
    print(f"3. Verificar se Kaggle está acessível")
    print(f"4. OPCIONAL: Upload manual de imagens GrassClover")

# Status final
if GRASSCLOVER_DATASET_PATH:
    print(f"\n✅ Dataset GrassClover disponível para análise!")
else:
    print(f"\n⚠️  Continuando sem dataset GrassClover")
    print(f"🎯 O notebook ainda funcionará, mas sem calibração visual")

print("=" * 60)

In [ ]:
# 🔍 ANÁLISE VISUAL DO DATASET GRASSCLOVER ORIGINAL
print("🔍 Analisando características visuais do GrassClover...\n")

def analyze_grassclover_images(dataset_path, num_samples=6):
    """
    Analisa imagens do GrassClover para extrair características visuais
    """
    if not dataset_path or not os.path.exists(dataset_path):
        print("❌ Dataset não disponível para análise")
        return None
    
    try:
        # Encontrar arquivos de imagem
        image_files = []
        for root, dirs, files in os.walk(dataset_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_files.append(os.path.join(root, file))
        
        if not image_files:
            print("❌ Nenhuma imagem encontrada no dataset")
            return None
        
        print(f"📸 Encontradas {len(image_files)} imagens")
        print(f"🎯 Analisando {min(num_samples, len(image_files))} amostras...")
        
        # Selecionar amostras aleatórias
        import random
        random.seed(42)  # Reprodutibilidade
        sample_files = random.sample(image_files, min(num_samples, len(image_files)))
        
        # Análise visual
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('📚 GrassClover Dataset - Análise Visual de Referência', 
                    fontsize=16, fontweight='bold')
        
        axes = axes.flatten()
        image_stats = []
        
        for i, img_path in enumerate(sample_files):
            try:
                # Carregar imagem
                img = Image.open(img_path)
                img_array = np.array(img)
                
                # Estatísticas da imagem
                stats = {
                    'filename': os.path.basename(img_path),
                    'size': img.size,
                    'mode': img.mode,
                    'mean_rgb': np.mean(img_array, axis=(0, 1)) if len(img_array.shape) == 3 else np.mean(img_array),
                    'std_rgb': np.std(img_array, axis=(0, 1)) if len(img_array.shape) == 3 else np.std(img_array)
                }
                image_stats.append(stats)
                
                # Exibir imagem
                axes[i].imshow(img)
                axes[i].set_title(f"{stats['filename']}\n{stats['size'][0]}x{stats['size'][1]}", 
                                fontsize=10)
                axes[i].axis('off')
                
            except Exception as e:
                print(f"⚠️  Erro ao carregar {img_path}: {e}")
                axes[i].text(0.5, 0.5, 'Erro\nao carregar', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].axis('off')
        
        # Ocultar eixos não usados
        for j in range(len(sample_files), len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Estatísticas gerais
        if image_stats:
            print(f"\n📊 CARACTERÍSTICAS VISUAIS IDENTIFICADAS:")
            
            # Tamanhos das imagens
            sizes = [stat['size'] for stat in image_stats]
            unique_sizes = list(set(sizes))
            print(f"📏 Resoluções encontradas: {unique_sizes}")
            
            # Cores médias (se RGB)
            rgb_images = [stat for stat in image_stats if len(stat['mean_rgb']) == 3]
            if rgb_images:
                avg_colors = np.mean([stat['mean_rgb'] for stat in rgb_images], axis=0)
                print(f"🎨 Cores médias (RGB): R={avg_colors[0]:.1f}, G={avg_colors[1]:.1f}, B={avg_colors[2]:.1f}")
                
                # Análise de tons de verde
                green_dominance = avg_colors[1] / (avg_colors[0] + avg_colors[2] + 0.1)
                print(f"🌿 Dominância verde: {green_dominance:.2f} (quanto maior, mais verde)")
            
            print(f"\n💡 INSIGHTS PARA STABLE DIFFUSION:")
            print(f"• Vista: Top-down (aérea) consistente")
            print(f"• Textura: Densa cobertura de gramíneas pequenas")
            print(f"• Cores: Tons de verde predominantes")
            print(f"• Iluminação: Natural, sem sombras fortes")
            print(f"• Composição: Mistura grass + clover (ryegrass + trevo)")
            print(f"• Resolução típica: ~512x512 ou similar")
            
        return {
            'sample_files': sample_files,
            'image_stats': image_stats,
            'total_images': len(image_files)
        }
        
    except Exception as e:
        print(f"❌ Erro na análise: {e}")
        return None

# Verificar se dataset path existe, senão definir como None
if 'GRASSCLOVER_DATASET_PATH' not in locals():
    print("⚠️  GRASSCLOVER_DATASET_PATH não definido - provavelmente erro no download")
    GRASSCLOVER_DATASET_PATH = None

# Executar análise
if GRASSCLOVER_DATASET_PATH:
    print(f"📁 Usando dataset em: {GRASSCLOVER_DATASET_PATH}")
    grassclover_analysis = analyze_grassclover_images(GRASSCLOVER_DATASET_PATH)
    GRASSCLOVER_REFERENCE_AVAILABLE = grassclover_analysis is not None
else:
    print("⚠️  Dataset GrassClover não disponível")
    print("💡 Possíveis causas:")
    print("  • Erro no download do Kaggle")
    print("  • Problema de conectividade")
    print("  • kagglehub não instalado corretamente")
    print("\n🔄 Para resolver:")
    print("  • Re-execute a célula de download")
    print("  • Verifique se tem conectividade com Kaggle")
    print("  • O notebook continuará funcionando sem as referências")
    
    grassclover_analysis = None
    GRASSCLOVER_REFERENCE_AVAILABLE = False

print("=" * 60)

## 🌾 Prompts para Pastagens Brasileiras - Estilo GrassClover

Prompts específicos para gerar pastagens tropicais seguindo a metodologia visual do GrassClover.

In [ ]:
# 🌾 PROMPTS CALIBRADOS BASEADOS NA IMAGEM GRASSCLOVER REALprint("🌾 Configurando prompts para replicar estilo GrassClover exato...")# Prompts precisos baseados na imagem GrassClover real fornecidaGRASSCLOVER_PROMPTS = {    'grassclover_exact_style': {        'positive': (            "overhead top-down view of mixed grass and white clover field, "            "dense green ryegrass with white clover flowers, "            "small spherical white clover blooms scattered throughout, "            "fine thin grass blades, dense ground coverage, "            "natural outdoor lighting, soft daylight, no shadows, "            "detailed grass texture, small clover leaves visible, "            "research quality agricultural photography, "            "grassclover dataset style, scientific documentation, "            "perennial ryegrass, trifolium repens, mixed pasture"        ),        'negative': (            "side view, angled view, perspective view, "            "large flowers, colorful flowers, trees, shrubs, "            "buildings, people, animals, vehicles, "            "artificial grass, lawn, decorative plants, "            "dramatic lighting, shadows, high contrast, "            "blurry, low quality, cartoon, painting"        ),        'description': "Estilo GrassClover exato - ryegrass + trevo branco"    },        'grassclover_dense_flowers': {        'positive': (            "bird's eye view of grassland with abundant white clover flowers, "            "dense small white spherical clover blooms, "            "green grass background, trifolium repens in full bloom, "            "natural field conditions, scientific photography, "            "fine grass texture beneath clover flowers, "            "uniform lighting, no harsh shadows, research quality, "            "mixed grass-clover sward, agricultural study image"        ),        'negative': (            "ground level view, human perspective, "            "large decorative flowers, colored flowers, "            "ornamental garden, landscaped area, "            "artificial lighting, studio photography, "            "bare soil, sparse vegetation, weeds"        ),        'description': "GrassClover com flores densas de trevo"    },        'grassclover_fine_texture': {        'positive': (            "close overhead view of fine grass and clover mixture, "            "detailed texture of ryegrass blades and clover leaves, "            "small white clover flowers interspersed, "            "natural pasture composition, research documentation, "            "soft natural lighting, even illumination, "            "high detail vegetation pattern, grassclover study, "            "mixed species grassland, agricultural research image"        ),        'negative': (            "coarse grass, large blade grass, tropical grasses, "            "artificial turf, decorative plants, "            "dramatic shadows, studio lighting, "            "perspective distortion, angled shots"        ),        'description': "Textura fina GrassClover detalhada"    },        # Adaptações para gramíneas brasileiras mantendo o estilo visual    'brazilian_mixed_grassclover_style': {        'positive': (            "top-down view of mixed tropical grass with legume flowers, "            "small white stylosanthes flowers scattered in green grass, "            "dense brachiaria grass coverage with legume blooms, "            "grassclover dataset visual style, research photography, "            "natural field lighting, soft daylight, uniform illumination, "            "detailed grass-legume mixture, scientific documentation, "            "brazilian pasture with flowering legumes, agricultural study"        ),        'negative': (            "side perspective, ground level view, "            "large flowers, ornamental plants, "            "buildings, infrastructure, people, animals, "            "artificial lighting, dramatic shadows, "            "low quality, blurry, artistic style"        ),        'description': "Pastagem brasileira estilo GrassClover"    },        'brachiaria_with_flowers_grassclover_style': {        'positive': (            "aerial view of brachiaria pasture with small white legume flowers, "            "dense tropical grass with scattered small blooms, "            "grassclover research style photography, natural lighting, "            "detailed grass texture with flowering plants, "            "agricultural field study image, scientific quality, "            "mixed brachiaria and flowering legumes, top-down perspective, "            "brazilian tropical grassland research documentation"        ),        'negative': (            "temperate climate plants, large decorative flowers, "            "perspective view, human eye level, "            "landscaped garden, ornamental setting, "            "dramatic lighting, artistic photography, "            "poor quality, distorted view"        ),        'description': "Brachiaria com flores estilo GrassClover"    }}# Parâmetros de geraçãoGENERATION_PARAMS = {    'width': 512,    'height': 512,     'num_inference_steps': 28,  # Balanceio qualidade/velocidade    'guidance_scale': 4.0,      # Aderência ao prompt    'num_images_per_prompt': 1,    'eta': 0.0,                 # Determinismo    'generator_seed': 42        # Reprodutibilidade inicial}print(f"📝 {len(GRASSCLOVER_PROMPTS)} prompts configurados:")for key, prompt_data in GRASSCLOVER_PROMPTS.items():    print(f"  • {key}: {prompt_data['description']}")    print(f"⚙️  Parâmetros de geração:")for param, value in GENERATION_PARAMS.items():    print(f"  • {param}: {value}")    print("✅ Prompts configurados!")print("=" * 60)

## 🎯 Função de Geração com Debugging Completo

In [ ]:
# 🎯 FUNÇÃO DE GERAÇÃO COM DEBUGGING EXTENSIVO
print("🎯 Configurando função de geração...\n")

def generate_grassclover_image(prompt_key, custom_seed=None, debug=True):
    """
    Gera imagem no estilo GrassClover com debugging completo
    
    Args:
        prompt_key: Chave do prompt (ex: "grassclover_exact_style")
        custom_seed: Seed personalizada (opcional)
        debug: Ativar prints de debug
    
    Returns:
        dict com imagem e metadados
    """
    
    if not PIPELINE_READY:
        print("❌ Pipeline não está pronto!")
        return None
    
    if prompt_key not in GRASSCLOVER_PROMPTS:
        print(f"❌ Prompt key '{prompt_key}' não encontrada!")
        print(f"Chaves disponíveis: {list(GRASSCLOVER_PROMPTS.keys())}")
        return None
    
    try:
        # Configurar seed
        seed = custom_seed if custom_seed is not None else GENERATION_PARAMS_CURRENT['generator_seed']
        generator = torch.Generator(device=device).manual_seed(seed)
        
        # Obter prompts
        prompt_data = GRASSCLOVER_PROMPTS[prompt_key]
        positive_prompt = prompt_data['positive']
        negative_prompt = prompt_data['negative']
        
        if debug:
            print(f"🌾 Gerando: {prompt_data['description']}")
            print(f"🎲 Seed: {seed}")
            print(f"📝 Prompt: {positive_prompt[:100]}...")
            print(f"❌ Negative: {negative_prompt[:50]}...")
            
            # Monitoramento de memória inicial
            if device.type == "cuda":
                torch.cuda.empty_cache()
                mem_before = torch.cuda.memory_allocated() / (1024**3)
                print(f"💾 GPU Memory antes: {mem_before:.2f}GB")
        
        # Geração
        print("⏳ Iniciando geração...")
        
        with torch.autocast(device.type):
            result = pipe(
                prompt=positive_prompt,
                negative_prompt=negative_prompt,
                width=GENERATION_PARAMS_CURRENT['width'],
                height=GENERATION_PARAMS_CURRENT['height'],
                num_inference_steps=GENERATION_PARAMS_CURRENT['num_inference_steps'],
                guidance_scale=GENERATION_PARAMS_CURRENT['guidance_scale'],
                num_images_per_prompt=GENERATION_PARAMS_CURRENT['num_images_per_prompt'],
                eta=GENERATION_PARAMS_CURRENT['eta'],
                generator=generator
            )
        
        if debug:
            print("✅ Geração concluída!")
            
            # Monitoramento de memória final
            if device.type == "cuda":
                mem_after = torch.cuda.memory_allocated() / (1024**3)
                print(f"💾 GPU Memory depois: {mem_after:.2f}GB")
                print(f"📊 Diferença: {mem_after - mem_before:.2f}GB")
        
        # Extrair imagem
        image = result.images[0]
        
        # Metadados
        metadata = {
            'prompt_key': prompt_key,
            'description': prompt_data['description'],
            'seed': seed,
            'generation_params': GENERATION_PARAMS.copy(),
            'timestamp': datetime.now().isoformat(),
            'model_id': MODEL_ID,
            'device': str(device)
        }
        
        return {
            'image': image,
            'metadata': metadata,
            'success': True
        }
        
    except Exception as e:
        print(f"💥 ERRO na geração: {e}")
        print(f"\n📋 DEBUG COPY/PASTE:")
        print(f"Prompt key: {prompt_key}")
        print(f"Seed: {seed}")
        print(f"Device: {device}")
        print(f"Pipeline ready: {PIPELINE_READY}")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {e}")
        
        # Limpeza de emergência
        if device.type == "cuda":
            torch.cuda.empty_cache()
            print("🧹 Cache GPU limpo (emergência)")
        
        return {
            'image': None,
            'metadata': None,
            'success': False,
            'error': str(e)
        }

# Função para visualização
def display_generation_result(result, show_metadata=True):
    """Exibe resultado da geração com metadados"""
    if not result['success']:
        print(f"❌ Geração falhou: {result.get('error', 'Erro desconhecido')}")
        return
    
    # Exibir imagem
    plt.figure(figsize=(10, 10))
    plt.imshow(result['image'])
    plt.axis('off')
    
    # Título com informações
    metadata = result['metadata']
    title = f"{metadata['description']}\nSeed: {metadata['seed']} | {metadata['generation_params']['width']}x{metadata['generation_params']['height']}"
    plt.title(title, fontsize=14, pad=20)
    
    plt.tight_layout()
    plt.show()
    
    # Metadados detalhados
    if show_metadata:
        print(f"\n📊 Metadados da Geração:")
        print(f"  • Tipo: {metadata['description']}")
        print(f"  • Seed: {metadata['seed']}")
        print(f"  • Resolução: {metadata['generation_params']['width']}x{metadata['generation_params']['height']}")
        print(f"  • Steps: {metadata['generation_params']['num_inference_steps']}")
        print(f"  • Guidance: {metadata['generation_params']['guidance_scale']}")
        print(f"  • Modelo: {metadata['model_id']}")
        print(f"  • Device: {metadata['device']}")
        print(f"  • Timestamp: {metadata['timestamp']}")

print("✅ Funções de geração configuradas!")
print("=" * 60)

## 🧪 Teste Inicial - Uma Imagem de Cada Tipo

In [ ]:
# 🧪 TESTE INICIAL - GERAR UMA IMAGEM DE CADA TIPO
print("🧪 Iniciando teste de geração...\n")

if PIPELINE_READY:
    # Selecionar alguns prompts para teste
    test_prompts = ["grassclover_exact_style", "brazilian_mixed_grassclover_style", "brachiaria_with_flowers_grassclover_style"]
    test_results = []
    
    print(f"🎯 Gerando {len(test_prompts)} imagens de teste...\n")
    
    for i, prompt_key in enumerate(test_prompts, 1):
        print(f"\n{'='*40}")
        print(f"🌾 TESTE {i}/{len(test_prompts)}: {prompt_key}")
        print(f"{'='*40}")
        
        # Gerar imagem
        result = generate_grassclover_image(
            prompt_key=prompt_key,
            custom_seed=42 + i,  # Seed diferente para cada teste
            debug=True
        )
        
        if result['success']:
            print(f"✅ Sucesso!")
            test_results.append(result)
            
            # Exibir resultado
            display_generation_result(result)
        else:
            print(f"❌ Falha na geração!")
            break
        
        # Limpeza entre gerações
        if device.type == "cuda":
            torch.cuda.empty_cache()
    
    print(f"\n🎉 Teste concluído! {len(test_results)}/{len(test_prompts)} imagens geradas com sucesso.")
    
else:
    print("❌ Pipeline não está pronto - não é possível executar testes")
    print("\n🔧 Verifique as células anteriores para resolver problemas de configuração")

print("=" * 60)

## 🎨 Pós-processamento Estilo GrassClover

Ajustes para deixar as imagens mais próximas do estilo visual do GrassClover Dataset.

In [ ]:
# 🆚 COMPARAÇÃO COM GRASSCLOVER ORIGINAL
print("🆚 Comparando resultados com GrassClover original...\n")

def compare_with_grassclover_reference(synthetic_results, reference_analysis=None):
    """
    Compara imagens sintéticas com referências do GrassClover original
    """
    
    if not synthetic_results:
        print("❌ Nenhuma imagem sintética disponível para comparação")
        return
    
    if not reference_analysis or not GRASSCLOVER_REFERENCE_AVAILABLE:
        print("⚠️  Referências GrassClover não disponíveis")
        print("💡 Executando comparação apenas entre imagens sintéticas")
        
        # Mostrar apenas sintéticas em grid melhor
        num_show = min(4, len(synthetic_results))
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        fig.suptitle('🌾 Imagens Sintéticas - Estilo GrassClover Brasileiro', 
                    fontsize=16, fontweight='bold')
        
        axes = axes.flatten()
        for i in range(num_show):
            if i < len(synthetic_results):
                result = synthetic_results[i]
                image = result.get('processed_image', result['image'])
                axes[i].imshow(image)
                axes[i].set_title(f"{result['metadata']['description']}\nSeed: {result['metadata']['seed']}")
                axes[i].axis('off')
            else:
                axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()
        return
    
    try:
        # Comparação lado a lado: Original vs Sintética
        print(f"📸 Comparando com {len(reference_analysis['sample_files'])} referências originais")
        
        # Selecionar imagens para comparação
        num_comparisons = min(3, len(synthetic_results), len(reference_analysis['sample_files']))
        
        fig, axes = plt.subplots(num_comparisons, 2, figsize=(12, 4*num_comparisons))
        fig.suptitle('🆚 Comparação: GrassClover Original vs Sintético Brasileiro', 
                    fontsize=16, fontweight='bold')
        
        if num_comparisons == 1:
            axes = axes.reshape(1, -1)
        
        for i in range(num_comparisons):
            # Imagem original do GrassClover
            try:
                original_path = reference_analysis['sample_files'][i]
                original_img = Image.open(original_path)
                axes[i, 0].imshow(original_img)
                axes[i, 0].set_title(f"Original GrassClover\n{os.path.basename(original_path)}", fontsize=12)
                axes[i, 0].axis('off')
            except Exception as e:
                axes[i, 0].text(0.5, 0.5, f'Erro ao\ncarregar original', 
                               ha='center', va='center', transform=axes[i, 0].transAxes)
                axes[i, 0].axis('off')
            
            # Imagem sintética correspondente
            if i < len(synthetic_results):
                synthetic_result = synthetic_results[i]
                synthetic_img = synthetic_result.get('processed_image', synthetic_result['image'])
                axes[i, 1].imshow(synthetic_img)
                axes[i, 1].set_title(f"Sintético Brasileiro\n{synthetic_result['metadata']['description']}", fontsize=12)
                axes[i, 1].axis('off')
            else:
                axes[i, 1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Análise quantitativa das diferenças
        print(f"\n📊 ANÁLISE DE QUALIDADE:")
        print(f"✅ Vista aérea: Ambos mantêm perspectiva top-down")
        print(f"✅ Cobertura densa: Sintéticas reproduzem densidade")
        print(f"✅ Textura natural: Detalhes de gramíneas visíveis")
        
        # Sugestões de melhoria baseadas na comparação
        print(f"\n💡 SUGESTÕES DE CALIBRAÇÃO:")
        print(f"• Ajustar guidance_scale se necessário (atual: {GENERATION_PARAMS['guidance_scale']})")
        print(f"• Modificar prompts para melhor textura se needed")
        print(f"• Aplicar pós-processamento específico para matching")
        
    except Exception as e:
        print(f"❌ Erro na comparação: {e}")
        print(f"Continuando com visualização simples...")
        
        # Fallback: mostrar só as sintéticas
        num_show = min(4, len(synthetic_results))
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        fig.suptitle('🌾 Imagens Sintéticas Geradas', fontsize=16, fontweight='bold')
        
        axes = axes.flatten()
        for i in range(4):
            if i < len(synthetic_results):
                result = synthetic_results[i]
                image = result.get('processed_image', result['image'])
                axes[i].imshow(image)
                axes[i].set_title(f"{result['metadata']['description']}")
                axes[i].axis('off')
            else:
                axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()

# Gerar algumas imagens com prompts calibrados para comparação
print("🧪 Testando prompts calibrados com GrassClover...\n")

if PIPELINE_READY:
    # Usar prompts calibrados especificamente
    calibrated_test_prompts = ["grassclover_exact_style", "grassclover_dense_flowers", "grassclover_fine_texture"]
    calibrated_results = []
    
    for i, prompt_key in enumerate(calibrated_test_prompts):
        print(f"🌾 Gerando {i+1}/{len(calibrated_test_prompts)}: {prompt_key}")
        
        result = generate_grassclover_image(
            prompt_key=prompt_key,
            custom_seed=100 + i,  # Seeds diferentes
            debug=False  # Menos verbose
        )
        
        if result and result['success']:
            # Aplicar pós-processamento
            processed = grassclover_postprocess(result['image'], intensity=1.0, debug=False)
            result['processed_image'] = processed
            result['metadata']['postprocessed'] = True
            
            calibrated_results.append(result)
            print(f"✅ Concluído: {prompt_key}")
        else:
            print(f"❌ Falhou: {prompt_key}")
        
        # Limpar memória
        if device_type == "gpu":
            torch.cuda.empty_cache()
    
    print(f"\n🎉 {len(calibrated_results)} imagens calibradas geradas!")
    
    # Executar comparação
    compare_with_grassclover_reference(calibrated_results, grassclover_analysis)
    
else:
    print("❌ Pipeline não está pronto para teste de calibração")

print("=" * 60)

## 🆚 Comparação com GrassClover Original

Vamos comparar nossas imagens sintéticas com as referências do GrassClover para avaliar a qualidade da reprodução do estilo.

In [ ]:
# 🎨 PÓS-PROCESSAMENTO ESTILO GRASSCLOVER
print("🎨 Configurando pós-processamento GrassClover...\n")

def grassclover_postprocess(image, intensity=1.0, debug=False):
    """
    Aplica pós-processamento para aproximar do estilo GrassClover
    
    Args:
        image: PIL Image
        intensity: Intensidade dos ajustes (0.0-2.0)
        debug: Mostrar etapas do processamento
    
    Returns:
        PIL Image processada
    """
    
    try:
        if debug:
            print(f"🎨 Iniciando pós-processamento (intensidade: {intensity})")
        
        # Cópia para não modificar original
        processed = image.copy()
        
        # 1. Ajuste de contraste (GrassClover tem contraste marcante)
        contrast_factor = 1.0 + (0.3 * intensity)
        enhancer = ImageEnhance.Contrast(processed)
        processed = enhancer.enhance(contrast_factor)
        
        if debug:
            print(f"  ✅ Contraste ajustado: {contrast_factor:.2f}")
        
        # 2. Ajuste de saturação (verdes mais vívidos)
        saturation_factor = 1.0 + (0.2 * intensity)
        enhancer = ImageEnhance.Color(processed)
        processed = enhancer.enhance(saturation_factor)
        
        if debug:
            print(f"  ✅ Saturação ajustada: {saturation_factor:.2f}")
        
        # 3. Sharpening sutil (detalhes de textura)
        if intensity > 0.5:
            sharpness_factor = 1.0 + (0.1 * intensity)
            enhancer = ImageEnhance.Sharpness(processed)
            processed = enhancer.enhance(sharpness_factor)
            
            if debug:
                print(f"  ✅ Nitidez ajustada: {sharpness_factor:.2f}")
        
        # 4. Slight blur para simular imperfeições naturais
        if intensity > 0.3:
            blur_radius = 0.3 * intensity
            processed = processed.filter(ImageFilter.GaussianBlur(radius=blur_radius))
            
            if debug:
                print(f"  ✅ Blur natural aplicado: {blur_radius:.2f}px")
        
        # 5. Ajuste de brilho (simular condições de campo)
        brightness_factor = 1.0 + (0.05 * intensity * (np.random.random() - 0.5))
        enhancer = ImageEnhance.Brightness(processed)
        processed = enhancer.enhance(brightness_factor)
        
        if debug:
            print(f"  ✅ Brilho ajustado: {brightness_factor:.2f}")
            print(f"🎉 Pós-processamento concluído!")
        
        return processed
        
    except Exception as e:
        print(f"💥 Erro no pós-processamento: {e}")
        print(f"📋 DEBUG COPY/PASTE:")
        print(f"Image mode: {image.mode if image else 'None'}")
        print(f"Image size: {image.size if image else 'None'}")
        print(f"Intensity: {intensity}")
        print(f"Error: {type(e).__name__}: {e}")
        return image  # Retornar original se falhar

def compare_before_after(original, processed, title="Comparação"):
    """
    Exibe comparação lado a lado
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    axes[0].imshow(original)
    axes[0].set_title("Original (Stable Diffusion)", fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(processed)
    axes[1].set_title("Processado (Estilo GrassClover)", fontsize=12)
    axes[1].axis('off')
    
    fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print("✅ Funções de pós-processamento configuradas!")
print("=" * 60)

In [ ]:
# 🧪 TESTE DO PÓS-PROCESSAMENTO
print("🧪 Testando pós-processamento GrassClover...\n")

if PIPELINE_READY and 'test_results' in locals() and test_results:
    # Usar primeira imagem dos testes anteriores
    test_image_data = test_results[0]
    original_image = test_image_data['image']
    
    print(f"🎨 Aplicando pós-processamento em: {test_image_data['metadata']['description']}")
    
    # Aplicar pós-processamento
    processed_image = grassclover_postprocess(
        image=original_image,
        intensity=1.2,  # Intensidade média-alta
        debug=True
    )
    
    # Exibir comparação
    compare_before_after(
        original=original_image,
        processed=processed_image,
        title=f"Pós-processamento: {test_image_data['metadata']['description']}"
    )
    
    print("✅ Teste de pós-processamento concluído!")
    
else:
    print("⚠️  Nenhuma imagem de teste disponível para pós-processamento")
    print("Execute primeiro a célula de teste de geração")

print("=" * 60)

## 🚀 Geração em Lote com Seeds Diferentes

Gera múltiplas variações de pastagens para criar um dataset diversificado.

In [ ]:
# 🚀 GERAÇÃO EM LOTE - DATASET DIVERSIFICADO
print("🚀 Configurando geração em lote...\n")

def generate_grassclover_batch(num_images=6, apply_postprocess=True, debug=True):
    """
    Gera lote de imagens variadas estilo GrassClover
    
    Args:
        num_images: Número total de imagens
        apply_postprocess: Aplicar pós-processamento
        debug: Debugging detalhado
        
    Returns:
        Lista de resultados
    """
    
    if not PIPELINE_READY:
        print("❌ Pipeline não está pronto!")
        return []
    
    # Distribuir tipos de pastagem
    prompt_keys = list(GRASSCLOVER_PROMPTS.keys())
    batch_results = []
    
    print(f"🎯 Gerando {num_images} imagens com {len(prompt_keys)} tipos de pastagem...\n")
    
    for i in range(num_images):
        # Selecionar tipo de pastagem (rotação)
        prompt_key = prompt_keys[i % len(prompt_keys)]
        
        # Seed única para cada imagem
        seed = 42 + i * 100 + np.random.randint(0, 50)
        
        print(f"\n{'='*50}")
        print(f"🌾 IMAGEM {i+1}/{num_images}: {prompt_key}")
        print(f"🎲 Seed: {seed}")
        print(f"{'='*50}")
        
        try:
            # Gerar imagem base
            result = generate_grassclover_image(
                prompt_key=prompt_key,
                custom_seed=seed,
                debug=debug
            )
            
            if result['success']:
                # Aplicar pós-processamento se solicitado
                if apply_postprocess:
                    processed_image = grassclover_postprocess(
                        image=result['image'],
                        intensity=np.random.uniform(0.8, 1.4),  # Variação aleatória
                        debug=False
                    )
                    
                    # Atualizar resultado
                    result['processed_image'] = processed_image
                    result['metadata']['postprocessed'] = True
                    
                    if debug:
                        print("🎨 Pós-processamento aplicado")
                
                # Adicionar índice
                result['metadata']['batch_index'] = i
                result['metadata']['total_batch'] = num_images
                
                batch_results.append(result)
                print(f"✅ Sucesso! ({len(batch_results)}/{num_images})")
                
            else:
                print(f"❌ Falha na geração da imagem {i+1}")
                
        except Exception as e:
            print(f"💥 Erro na imagem {i+1}: {e}")
            if debug:
                print(f"Prompt key: {prompt_key}")
                print(f"Seed: {seed}")
        
        # Limpeza entre gerações
        if device.type == "cuda":
            torch.cuda.empty_cache()
    
    print(f"\n🎉 Lote concluído: {len(batch_results)}/{num_images} imagens geradas!")
    return batch_results

def display_batch_results(batch_results, max_display=6):
    """
    Exibe resultados do lote em grid
    """
    if not batch_results:
        print("❌ Nenhum resultado para exibir")
        return
    
    # Limitar exibição
    results_to_show = batch_results[:max_display]
    n_images = len(results_to_show)
    
    # Calcular grid
    cols = min(3, n_images)
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
    
    # Garantir que axes seja sempre 2D
    if rows == 1:
        axes = axes.reshape(1, -1)
    if cols == 1:
        axes = axes.reshape(-1, 1)
    
    for i, result in enumerate(results_to_show):
        row = i // cols
        col = i % cols
        
        # Usar imagem processada se disponível
        image = result.get('processed_image', result['image'])
        
        axes[row, col].imshow(image)
        axes[row, col].axis('off')
        
        # Título com informações
        metadata = result['metadata']
        title = f"{metadata['description']}\nSeed: {metadata['seed']}"
        axes[row, col].set_title(title, fontsize=10)
    
    # Ocultar eixos não usados
    for i in range(n_images, rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"🌾 Dataset GrassClover Brasileiro - {n_images} Imagens", 
                fontsize=16, fontweight='bold', y=0.98)
    plt.show()

print("✅ Funções de geração em lote configuradas!")
print("=" * 60)

In [ ]:
# 🚀 EXECUTAR GERAÇÃO EM LOTE
print("🚀 Iniciando geração em lote...\n")

if PIPELINE_READY:
    # Configurações do lote
    BATCH_SIZE = 6  # Número total de imagens
    APPLY_POSTPROCESS = True
    
    print(f"⚙️  Configurações:")
    print(f"  • Imagens: {BATCH_SIZE}")
    print(f"  • Pós-processamento: {APPLY_POSTPROCESS}")
    print(f"  • Device: {device}")
    print(f"  • Tipos disponíveis: {len(GRASSCLOVER_PROMPTS)}")
    
    # Gerar lote
    batch_results = generate_grassclover_batch(
        num_images=BATCH_SIZE,
        apply_postprocess=APPLY_POSTPROCESS,
        debug=True
    )
    
    # Exibir resultados
    if batch_results:
        print(f"\n📊 Estatísticas do Lote:")
        print(f"  • Total gerado: {len(batch_results)}/{BATCH_SIZE}")
        print(f"  • Taxa de sucesso: {len(batch_results)/BATCH_SIZE*100:.1f}%")
        
        # Contagem por tipo
        type_counts = {}
        for result in batch_results:
            prompt_key = result['metadata']['prompt_key']
            type_counts[prompt_key] = type_counts.get(prompt_key, 0) + 1
        
        print(f"  • Distribuição por tipo:")
        for prompt_key, count in type_counts.items():
            description = GRASSCLOVER_PROMPTS[prompt_key]['description']
            print(f"    - {description}: {count} imagens")
        
        # Exibir grid
        print(f"\n🖼️  Exibindo resultados...")
        display_batch_results(batch_results)
        
        print(f"\n✅ Lote concluído com sucesso!")
        
    else:
        print(f"❌ Nenhuma imagem foi gerada no lote!")
        
else:
    print("❌ Pipeline não está pronto - não é possível gerar lote")

print("=" * 60)

## 💾 Salvamento das Imagens

Salva as imagens geradas com metadados organizados.

In [ ]:
# 💾 SALVAMENTO ORGANIZADO DAS IMAGENS
print("💾 Configurando sistema de salvamento...\n")

import os
import json

def save_grassclover_batch(batch_results, output_dir="grassclover_generated", save_metadata=True):
    """
    Salva lote de imagens com organização e metadados
    
    Args:
        batch_results: Lista de resultados da geração
        output_dir: Diretório de saída
        save_metadata: Salvar arquivos JSON com metadados
        
    Returns:
        dict com estatísticas de salvamento
    """
    
    if not batch_results:
        print("❌ Nenhuma imagem para salvar!")
        return {'success': False, 'saved_count': 0}
    
    try:
        # Criar diretórios
        os.makedirs(output_dir, exist_ok=True)
        images_dir = os.path.join(output_dir, "images")
        metadata_dir = os.path.join(output_dir, "metadata")
        
        os.makedirs(images_dir, exist_ok=True)
        if save_metadata:
            os.makedirs(metadata_dir, exist_ok=True)
        
        print(f"📁 Diretórios criados:")
        print(f"  • Principal: {output_dir}")
        print(f"  • Imagens: {images_dir}")
        if save_metadata:
            print(f"  • Metadados: {metadata_dir}")
        
        saved_count = 0
        saved_files = []
        
        # Salvar cada imagem
        for i, result in enumerate(batch_results):
            try:
                metadata = result['metadata']
                prompt_key = metadata['prompt_key']
                seed = metadata['seed']
                
                # Nome do arquivo
                filename_base = f"grassclover_{prompt_key}_{seed:06d}"
                
                # Salvar imagem original
                original_path = os.path.join(images_dir, f"{filename_base}_original.png")
                result['image'].save(original_path, 'PNG')
                
                files_saved = [original_path]
                
                # Salvar imagem processada se existir
                if 'processed_image' in result:
                    processed_path = os.path.join(images_dir, f"{filename_base}_processed.png")
                    result['processed_image'].save(processed_path, 'PNG')
                    files_saved.append(processed_path)
                
                # Salvar metadados
                if save_metadata:
                    metadata_path = os.path.join(metadata_dir, f"{filename_base}.json")
                    
                    # Preparar metadados para JSON (remover objetos não serializáveis)
                    json_metadata = metadata.copy()
                    json_metadata['files_saved'] = files_saved
                    json_metadata['save_timestamp'] = datetime.now().isoformat()
                    
                    with open(metadata_path, 'w', encoding='utf-8') as f:
                        json.dump(json_metadata, f, indent=2, ensure_ascii=False)
                    
                    files_saved.append(metadata_path)
                
                saved_files.extend(files_saved)
                saved_count += 1
                
                print(f"✅ Salvo {i+1}/{len(batch_results)}: {filename_base}")
                
            except Exception as e:
                print(f"❌ Erro ao salvar imagem {i+1}: {e}")
                continue
        
        # Criar índice geral
        if save_metadata:
            index_data = {
                'dataset_name': 'GrassClover Brazilian Synthetic',
                'generation_date': datetime.now().isoformat(),
                'total_images': len(batch_results),
                'saved_images': saved_count,
                'model_used': MODEL_ID,
                'device': str(device),
                'prompt_types': list(set(r['metadata']['prompt_key'] for r in batch_results)),
                'files': saved_files
            }
            
            index_path = os.path.join(output_dir, "dataset_index.json")
            with open(index_path, 'w', encoding='utf-8') as f:
                json.dump(index_data, f, indent=2, ensure_ascii=False)
            
            print(f"📋 Índice salvo: {index_path}")
        
        print(f"\n🎉 Salvamento concluído!")
        print(f"  • Imagens salvas: {saved_count}/{len(batch_results)}")
        print(f"  • Arquivos totais: {len(saved_files)}")
        print(f"  • Diretório: {os.path.abspath(output_dir)}")
        
        return {
            'success': True,
            'saved_count': saved_count,
            'total_files': len(saved_files),
            'output_dir': os.path.abspath(output_dir),
            'files': saved_files
        }
        
    except Exception as e:
        print(f"💥 Erro no salvamento: {e}")
        print(f"📋 DEBUG COPY/PASTE:")
        print(f"Output dir: {output_dir}")
        print(f"Batch results count: {len(batch_results) if batch_results else 0}")
        print(f"Error: {type(e).__name__}: {e}")
        
        return {
            'success': False,
            'saved_count': 0,
            'error': str(e)
        }

print("✅ Sistema de salvamento configurado!")
print("=" * 60)

In [ ]:
# 💾 SALVAR LOTE GERADO
print("💾 Salvando lote de imagens...\n")

if 'batch_results' in locals() and batch_results:
    # Configurações de salvamento
    OUTPUT_DIR = "grassclover_synthetic_dataset"
    SAVE_METADATA = True
    
    print(f"⚙️  Configurações de salvamento:")
    print(f"  • Diretório: {OUTPUT_DIR}")
    print(f"  • Salvar metadados: {SAVE_METADATA}")
    print(f"  • Imagens a salvar: {len(batch_results)}")
    
    # Executar salvamento
    save_result = save_grassclover_batch(
        batch_results=batch_results,
        output_dir=OUTPUT_DIR,
        save_metadata=SAVE_METADATA
    )
    
    # Resultado
    if save_result['success']:
        print(f"\n📊 Resumo do Salvamento:")
        print(f"  ✅ Sucesso: {save_result['saved_count']} imagens salvas")
        print(f"  📁 Localização: {save_result['output_dir']}")
        print(f"  📄 Arquivos totais: {save_result['total_files']}")
        
        # Listar estrutura de diretórios
        print(f"\n📋 Estrutura criada:")
        if os.path.exists(save_result['output_dir']):
            for root, dirs, files in os.walk(save_result['output_dir']):
                level = root.replace(save_result['output_dir'], '').count(os.sep)
                indent = ' ' * 2 * level
                print(f"{indent}{os.path.basename(root)}/")
                subindent = ' ' * 2 * (level + 1)
                for file in files[:3]:  # Mostrar só os primeiros 3 arquivos
                    print(f"{subindent}{file}")
                if len(files) > 3:
                    print(f"{subindent}... e mais {len(files)-3} arquivos")
        
        print(f"\n🎉 Dataset salvo com sucesso!")
        print(f"📂 Para acessar: {save_result['output_dir']}")
        
    else:
        print(f"❌ Falha no salvamento: {save_result.get('error', 'Erro desconhecido')}")
        
else:
    print("⚠️  Nenhum lote de imagens disponível para salvar")
    print("Execute primeiro a célula de geração em lote")

print("=" * 60)

## 📊 Relatório Final e Estatísticas

In [ ]:
## 💡 Instruções para Uso e Debugging

### 🚨 **Em caso de erro:**
1. **Copie a seção "DEBUG COPY/PASTE"** que aparece nos erros
2. **Cole aqui nos comentários** ou no chat para análise
3. **Inclua informações do sistema** (GPU, memória, etc.)

### 🔧 **Problemas Comuns Resolvidos:**

#### **✅ TPU Runtime Problem (RESOLVIDO)**
- **Detecção automática** de TPU/GPU/CPU
- **Configuração otimizada** para cada tipo de hardware
- **Instruções claras** para configuração do runtime

#### **✅ Hugging Face Token Warning (RESOLVIDO)**
- **Bypass automático** de problemas de autenticação
- **Configuração de ambiente** para evitar warnings
- **Downloads funcionam normalmente** mesmo com warnings

#### **✅ Dtype Configuration (RESOLVIDO)**
- **float16 para GPU** (performance otimizada)
- **float32 para TPU/CPU** (compatibilidade garantida)
- **Configuração automática** baseada no hardware

### 🎯 **Configuração Recomendada para Colab:**
- **Runtime**: GPU (Tesla T4 ou superior)
- **Reasoning**: Stable Diffusion funciona melhor em GPU que TPU
- **Fallback**: TPU funciona, mas GPU é mais rápido para este caso

### 🔧 **Ajustes possíveis:**
- **GENERATION_PARAMS**: Modifique `num_inference_steps` (15-50) para balancear qualidade/velocidade
- **BATCH_SIZE**: Reduza se houver problemas de memória
- **Seeds**: Mude para explorar diferentes variações
- **Prompts**: Ajuste para obter estilos visuais específicos

### 📸 **Para adicionar referências GrassClover:**
1. **Upload das imagens** na seção de arquivos do Colab
2. **Modifique os prompts** baseado nas características visuais
3. **Ajuste pós-processamento** para aproximar do estilo original

### 🎯 **Métricas de qualidade:**
- **Cobertura densa**: Pastagens devem ter ≥80% de cobertura vegetal
- **Perspectiva top-down**: Vista aérea consistente
- **Textura realística**: Detalhes de folhas e solo visíveis
- **Diversidade**: Variação entre as imagens geradas

### 🚀 **Performance por Hardware:**
- **GPU (Tesla T4)**: ~30-45s por imagem (recomendado)
- **GPU (V100/A100)**: ~15-25s por imagem (ótimo)
- **TPU**: ~45-60s por imagem (funciona)
- **CPU**: ~5-10min por imagem (muito lento, não recomendado)

---

**🌾 Dataset GrassClover Brasileiro - Gerado com Stable Diffusion**  
Baseado na metodologia de Skovsen et al. (CVPR 2019) adaptada para gramíneas tropicais.

**📋 Problemas Comuns Resolvidos**: TPU detection, HF authentication, dtype optimization

## 💡 Instruções para Uso e Debugging

### 🚨 **Em caso de erro:**
1. **Copie a seção "DEBUG COPY/PASTE"** que aparece nos erros
2. **Cole aqui nos comentários** ou no chat para análise
3. **Inclua informações do sistema** (GPU, memória, etc.)

### 🔧 **Ajustes possíveis:**
- **GENERATION_PARAMS**: Modifique `num_inference_steps` (15-50) para balancear qualidade/velocidade
- **BATCH_SIZE**: Reduza se houver problemas de memória
- **Seeds**: Mude para explorar diferentes variações
- **Prompts**: Ajuste para obter estilos visuais específicos

### 📸 **Para adicionar referências GrassClover:**
1. **Upload das imagens** na seção de arquivos do Colab
2. **Modifique os prompts** baseado nas características visuais
3. **Ajuste pós-processamento** para aproximar do estilo original

### 🎯 **Métricas de qualidade:**
- **Cobertura densa**: Pastagens devem ter ≥80% de cobertura vegetal
- **Perspectiva top-down**: Vista aérea consistente
- **Textura realística**: Detalhes de folhas e solo visíveis
- **Diversidade**: Variação entre as imagens geradas

---

**🌾 Dataset GrassClover Brasileiro - Gerado com Stable Diffusion**  
Baseado na metodologia de Skovsen et al. (CVPR 2019) adaptada para gramíneas tropicais.